[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shang-vikas/series1-coding-exercises/blob/main/exercises/blog-06/exercise-00.ipynb)

# 🧪 Exercise — IMDB Sentiment with LSTM & GRU

You already trained a vanilla RNN and saw:

- Works decently
- Gradients decay
- Long reviews hurt
- Sequential bottleneck remains



- Replace RNN → LSTM
- Replace RNN → GRU
- Compare performance
- Visualize gradients again
- Measure speed
- Discuss real tradeoffs

All on the same IMDB pipeline.

No theory fluff. Just architecture evolution you can feel.

We reuse dataset pipeline.

Only model changes.

## 🔹 1️⃣ Setup & Data Loading

In [1]:
%pip install datasets -q

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from datasets import load_dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from collections import Counter
import time
import matplotlib.pyplot as plt
import re

# Load IMDB dataset from Hugging Face datasets
dataset = load_dataset("imdb")
train_data = dataset['train']
test_data = dataset['test']

print("Train samples:", len(train_data))
print("Test samples:", len(test_data))

# Tokenization
def basic_english_tokenizer(text):
    """Simple tokenizer that splits on whitespace and converts to lowercase."""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    return text.split()

tokenizer = basic_english_tokenizer

# Build vocabulary
counter = Counter()
for example in train_data:
    text = example['text']
    tokens = tokenizer(text)
    counter.update(tokens)

vocab_size = 20000
most_common = counter.most_common(vocab_size - 2)
vocab = {word: idx+2 for idx, (word, _) in enumerate(most_common)}
vocab["<pad>"] = 0
vocab["<unk>"] = 1

def encode(text):
    tokens = tokenizer(text)
    return [vocab.get(token, vocab["<unk>"]) for token in tokens]

def collate_batch(batch):
    texts, labels = [], []
    for example in batch:
        text = example['text']
        label = example['label']  # Already 0 or 1
        encoded = torch.tensor(encode(text))
        texts.append(encoded)
        labels.append(label)

    # Get lengths before padding
    lengths = torch.tensor([len(text) for text in texts], dtype=torch.long)

    texts = pad_sequence(texts, batch_first=True)
    labels = torch.tensor(labels)

    return texts, lengths, labels

train_loader = DataLoader(train_data, batch_size=64, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_data, batch_size=64, collate_fn=collate_batch)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Train samples: 25000
Test samples: 25000
Using device: cuda


## 🔹 2️⃣ Vanilla RNN Model (Baseline)

In [2]:
class VanillaRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, lengths):
        x = self.embedding(x)

        # Pack padded sequence
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)

        out, hidden = self.rnn(packed)
        final_hidden = hidden.squeeze(0)
        return self.fc(final_hidden)

## 🔹 3️⃣ LSTM Model

In [3]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, lengths):
        x = self.embedding(x)

        # Pack padded sequence
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)

        out, (h_n, c_n) = self.lstm(packed)

        final_hidden = h_n[-1]
        return self.fc(final_hidden)

## 🔹 4️⃣ GRU Model

In [4]:
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, lengths):
        x = self.embedding(x)

        # Pack padded sequence
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)

        out, h_n = self.gru(packed)

        final_hidden = h_n[-1]
        return self.fc(final_hidden)

## 🔹 5️⃣ Training & Evaluation Functions

In [5]:

def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for texts, lengths, labels in test_loader:
            texts, lengths, labels = texts.to(device), lengths.to(device), labels.to(device)
            outputs = torch.sigmoid(model(texts, lengths).view(-1))
            preds = (outputs > 0.5).long()

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total

def train_model(model, epochs=5):
    model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        start = time.time()

        for texts, lengths, labels in train_loader:
            texts, lengths, labels = texts.to(device), lengths.to(device), labels.to(device).float()

            outputs = model(texts, lengths).view(-1)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        acc = evaluate_model(model)
        print(f"Epoch {epoch+1} | Loss {total_loss/len(train_loader):.4f} | Acc {acc:.4f} | Time {time.time()-start:.2f}s")

In [6]:
rnn_model = VanillaRNN(vocab_size, 100, 128)
lstm_model = LSTMModel(vocab_size, 100, 128)
gru_model = GRUModel(vocab_size, 100, 128)

print("=" * 60)
print("Training RNN")
print("=" * 60)
train_model(rnn_model)

print("\n" + "=" * 60)
print("Training LSTM")
print("=" * 60)
train_model(lstm_model)

print("\n" + "=" * 60)
print("Training GRU")
print("=" * 60)
train_model(gru_model)

Training RNN
Epoch 1 | Loss 0.6590 | Acc 0.5860 | Time 32.72s
Epoch 2 | Loss 0.6143 | Acc 0.6814 | Time 28.45s
Epoch 3 | Loss 0.5341 | Acc 0.7031 | Time 30.34s
Epoch 4 | Loss 0.5141 | Acc 0.7596 | Time 27.86s
Epoch 5 | Loss 0.5406 | Acc 0.6624 | Time 27.71s

Training LSTM
Epoch 1 | Loss 0.5911 | Acc 0.7270 | Time 30.21s
Epoch 2 | Loss 0.4736 | Acc 0.5028 | Time 28.88s
Epoch 3 | Loss 0.5569 | Acc 0.6499 | Time 28.60s
Epoch 4 | Loss 0.3757 | Acc 0.8293 | Time 28.68s
Epoch 5 | Loss 0.3112 | Acc 0.8634 | Time 28.36s

Training GRU
Epoch 1 | Loss 0.6005 | Acc 0.6355 | Time 28.35s
Epoch 2 | Loss 0.5062 | Acc 0.8236 | Time 28.28s
Epoch 3 | Loss 0.4333 | Acc 0.8209 | Time 28.34s
Epoch 4 | Loss 0.2743 | Acc 0.8709 | Time 29.15s
Epoch 5 | Loss 0.1972 | Acc 0.8834 | Time 27.98s


## 🔍 Expected Results

| Model | Accuracy | Training Speed |
|-------|----------|----------------|
| RNN   | ~75-85%  | Slow           |
| LSTM  | ~80-89%  | Fast         |
| GRU   | ~85-88%  | Slightly faster than LSTM |

Numbers vary — but pattern holds.

## 🔬 7️⃣ Compare Gradient Flow

Reuse earlier gradient visualization.

Modify models to save outputs for gradient inspection:

In [7]:
class LSTMModelGrad(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, lengths):
        x = self.embedding(x)

        # Pack padded sequence
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)

        out, (h_n, c_n) = self.lstm(packed)

        # Unpack for gradient inspection (we need the full sequence)
        out_unpacked, _ = pad_packed_sequence(out, batch_first=True)
        out_unpacked.retain_grad()
        self.saved_outputs = out_unpacked

        final_hidden = h_n[-1]
        return self.fc(final_hidden)

class GRUModelGrad(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, lengths):
        x = self.embedding(x)

        # Pack padded sequence
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)

        out, h_n = self.gru(packed)

        # Unpack for gradient inspection (we need the full sequence)
        out_unpacked, _ = pad_packed_sequence(out, batch_first=True)
        out_unpacked.retain_grad()
        self.saved_outputs = out_unpacked

        final_hidden = h_n[-1]
        return self.fc(final_hidden)

class VanillaRNNGrad(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, lengths):
        x = self.embedding(x)

        # Pack padded sequence
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)

        out, hidden = self.rnn(packed)

        # Unpack for gradient inspection (we need the full sequence)
        out_unpacked, _ = pad_packed_sequence(out, batch_first=True)
        out_unpacked.retain_grad()
        self.saved_outputs = out_unpacked

        final_hidden = hidden.squeeze(0)
        return self.fc(final_hidden)

In [8]:
def visualize_gradient_decay(model, loader, model_name):
    model.train()
    criterion = nn.BCEWithLogitsLoss()

    texts, lengths, labels = next(iter(loader))
    texts, lengths, labels = texts.to(device), lengths.to(device), labels.to(device).float()

    outputs = model(texts, lengths).view(-1)
    loss = criterion(outputs, labels)

    model.zero_grad()
    loss.backward()

    # Get gradients w.r.t. hidden outputs
    grads = model.saved_outputs.grad

    if grads is None:
        print(f"{model_name}: Enable requires_grad for saved_outputs")
        return None

    # Average gradient magnitude per timestep
    grad_magnitudes = grads.abs().mean(dim=(0,2)).detach().cpu().numpy()

    plt.figure(figsize=(10, 6))
    plt.plot(grad_magnitudes, label=model_name)
    plt.title(f"Gradient Magnitude Across Time Steps - {model_name}")
    plt.xlabel("Time Step")
    plt.ylabel("Average Gradient Magnitude")
    plt.legend()
    plt.grid(True)
    plt.show()

    return grad_magnitudes

In [9]:
# Initialize models for gradient inspection
rnn_grad = VanillaRNNGrad(vocab_size, 100, 128).to(device)
lstm_grad = LSTMModelGrad(vocab_size, 100, 128).to(device)
gru_grad = GRUModelGrad(vocab_size, 100, 128).to(device)

print("RNN Gradient Flow:")
rnn_grads = visualize_gradient_decay(rnn_grad, train_loader, "RNN")

print("\nLSTM Gradient Flow:")
lstm_grads = visualize_gradient_decay(lstm_grad, train_loader, "LSTM")

print("\nGRU Gradient Flow:")
gru_grads = visualize_gradient_decay(gru_grad, train_loader, "GRU")

RNN Gradient Flow:
RNN: Enable requires_grad for saved_outputs

LSTM Gradient Flow:
LSTM: Enable requires_grad for saved_outputs

GRU Gradient Flow:
GRU: Enable requires_grad for saved_outputs


You'll notice:

- **RNN** → steep decay
- **LSTM** → flatter curve
- **GRU** → similar but slightly noisier

That's gated memory protecting gradient.

## 🕒 8️⃣ Measure Serial Bottleneck

Time per epoch:

Increase max sequence length.

RNN, LSTM, GRU all slow down similarly.

Because:

- All are sequential.
- None parallelize across time.

That bottleneck survives.

In [10]:
def collate_batch_long(batch, max_len=400):
    texts, labels = [], []
    for example in batch:
        text = example['text']
        label = example['label']  # Already 0 or 1
        encoded = torch.tensor(encode(text)[:max_len])
        texts.append(encoded)
        labels.append(label)

    # Get lengths before padding
    lengths = torch.tensor([len(text) for text in texts], dtype=torch.long)

    texts = pad_sequence(texts, batch_first=True)
    labels = torch.tensor(labels)

    return texts, lengths, labels

# Test with different sequence lengths
for max_len in [100, 200, 400]:
    print(f"\n{'='*60}")
    print(f"Testing with max sequence length: {max_len}")
    print(f"{'='*60}")

    train_loader_test = DataLoader(
        train_data[:1000],  # subset for speed
        batch_size=32,
        shuffle=False,
        collate_fn=lambda b: collate_batch_long(b, max_len)
    )

    # Test RNN
    rnn_test = VanillaRNN(vocab_size, 100, 128).to(device)
    start = time.time()
    for texts, lengths, labels in train_loader_test:
        texts, lengths, labels = texts.to(device), lengths.to(device), labels.to(device).float()
        _ = rnn_test(texts, lengths)
    rnn_time = time.time() - start

    # Test LSTM
    lstm_test = LSTMModel(vocab_size, 100, 128).to(device)
    start = time.time()
    for texts, lengths, labels in train_loader_test:
        texts, lengths, labels = texts.to(device), lengths.to(device), labels.to(device).float()
        _ = lstm_test(texts, lengths)
    lstm_time = time.time() - start

    # Test GRU
    gru_test = GRUModel(vocab_size, 100, 128).to(device)
    start = time.time()
    for texts, lengths, labels in train_loader_test:
        texts, lengths, labels = texts.to(device), lengths.to(device), labels.to(device).float()
        _ = gru_test(texts, lengths)
    gru_time = time.time() - start

    print(f"RNN:  {rnn_time:.2f}s")
    print(f"LSTM: {lstm_time:.2f}s")
    print(f"GRU:  {gru_time:.2f}s")
    print(f"All scale roughly linearly with sequence length")

## 🧠 What Improved Over RNN?

### ✅ Better long-term memory

Earlier tokens influence prediction more reliably.

### ✅ More stable gradients

Less vanishing.

### ✅ Higher accuracy

Especially on longer reviews.

## ⚠️ But They Still Have Limits

### ❌ Still sequential

Time step t must finish before t+1.

GPU underutilized.

### ❌ Still compress entire history

Final hidden state is fixed size.

Information loss still exists.

### ❌ Still struggle with very long dependencies

Improved ≠ solved.

## 🧩 When GRU vs LSTM?

### LSTM Pros

- Slightly more expressive
- Better when data is complex

### LSTM Cons

- More parameters
- Slower

### GRU Pros

- Simpler
- Faster
- Often similar accuracy

### GRU Cons

- Slightly less expressive
- Sometimes underperforms on very complex tasks

## 🧨 The Structural Ceiling

Even with gates:

```
Token1 → Token2 → Token3 → ... → TokenN
```

Information still walks step by step.

Memory still compressed.

Gradient still multiplied through time.

That's the real limitation.

## 🧠 Why This Matters Before Transformers

You now saw:

- **RNN** → works but fragile
- **LSTM/GRU** → stabilizes memory
- But sequential nature remains

So the next question becomes inevitable:

**What if we removed recurrence entirely?**

And that's where attention and Transformers enter.